<a href="https://colab.research.google.com/github/RajKGupta/MIDAS_Labs/blob/master/MIDAS_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tweepy
import json
import pandas as pd
from IPython.display import display, HTML

In [0]:
pd.set_option('display.max_colwidth', -1)

In [0]:
# the original keys has been replaced by dummies
consumer_key = 'dummy'
consumer_secret = 'dummy'
access_key = 'dummy'
access_secret = 'dummy'

def get_all_tweets(screen_name):
	auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
	auth.set_access_token(access_key, access_secret) #auth
	api = tweepy.API(auth)
	alltweets = []	
	new_tweets = api.user_timeline(screen_name = screen_name,count=199, tweet_mode='extended') # 199 tweets are being taken from the tiemeline in one fetch
	alltweets.extend(new_tweets)
	oldest = alltweets[-1].id - 1
	while len(new_tweets) > 0:
		new_tweets = api.user_timeline(screen_name = screen_name,count=199,max_id=oldest, tweet_mode='extended')
		alltweets.extend(new_tweets)  # the alltweet array is extended to incorporate the new tweets in the next fetch
		oldest = alltweets[-1].id - 1
	return alltweets

def store_tweets(alltweets, file):  # function to format the data in a suitable json compatible structure and then dumping the data in a file
	tweet_list=[]
	for tweet in alltweets:
		midasDict=dict()
		midasDict['text']=tweet.full_text
		midasDict['created_at']=tweet.created_at.strftime("%Y-%m-%d %H:%M:%S")
		midasDict['retweet_count']=tweet.retweet_count
		midasDict['likes']=tweet.favorite_count 
		c = 0
    
    # logic to handle the count of photos
		try:
			if tweet.extended_entities:
				for i in  (tweet.extended_entities['media']):
					if i['type'] == 'photo':
						c+=1
		except:
			c = 'None';
		midasDict['image_count']=c
		tweet_list.append(midasDict)
	with open(file, 'w') as outfile:
		json.dump(tweet_list, outfile, indent=4, sort_keys=True)

def retrieve_tweets(file):  # function to retreive the data from the dumped json file
  with open(file, 'r') as infile:
    data = json.load(infile)
    df = pd.DataFrame.from_records(data)
    display(HTML(df.to_html(justify = 'unset').replace('\\n', ' '))) # New Lines not displayed properly due to right alignment of DataFrame
    
    

In [0]:
file='tweets.json'

In [0]:
alltweets = get_all_tweets('midasIIITD')  # fetching tweets from the target's timeline in this case Twitter handle @midasIIITD

In [0]:
store_tweets(alltweets, file) # dumping the tweets

In [57]:
retrieve_tweets(file) # retreiveing the tweet from the stored file

,created_at,image_count,likes,retweet_count,text
0,2019-04-08 07:08:12,1,12,2,"Many Congratulations to @midasIIITD student, Shagun Uppal @shagunuppls, on getting selected for the summer internship in the BRAIN lab at Singapore University of Technology &amp; Design @sutdsg, Singapore. We wish her the best luck for the internship. #MIDAS #StudentAchievement https://t.co/snX2GkzvQg"
1,2019-04-08 03:27:42,1,3,0,@midasIIITD thanks all students who have appeared for the interview yesterday. We will announce the interview results for MIDAS internship latest by 14th April 2019. Sorry for the delay since we are carefully evaluating all applications that we have received for the internship. https://t.co/FANz5vImwU
2,2019-04-07 14:17:29,None,0,0,"@himanchalchandr Meanwhile, complete CV/NLP task first."
3,2019-04-07 14:17:09,None,0,0,@sayangdipto123 Submit as per the guideline again.
4,2019-04-07 11:43:24,None,1,1,We request all students whose interview are scheduled today to join at the time given to them and not before or after. We have other interviews scheduled which will be affected. Thanks much.
5,2019-04-07 06:55:19,None,5,2,"Other queries: ""none of the Tweeter Apis give the correct count of favorites tested for most of them, all give the wrong count. same is true for retweet. this mostly happens if the no. of likes, retweet is very large. So, what shld be done?"" Ans: Just use the count given by API."
6,2019-04-07 06:53:38,None,4,1,"Other queries: ""do we have to make two different repositories on Github as ""Python Problem"" and ""CV Problem"". Or we have to make a single repository named ""root"" and upload our code in that?"" And: Have one repository of any name which has two folders. One python and other cv/nlp"
7,2019-04-07 05:32:27,None,6,1,"Other queries: ""If using Twitter api, it does not include any information about images in the response objects sent, so how do we retrieve the image info as requested?"" Ans: U have to use twitter api or wrappers based on it like tweepy. Twitter API does provide image information."
8,2019-04-07 05:29:40,None,7,1,"Response to some queries asked by students on @midasIIITD task. What does the line ""dump the responses into JSONlines file"" mean? It means you have to dump the response into a jsonlines file. jsonlines is a file extension more popularly know as .jsonl #MIDAS #Research #Internship"
9,2019-04-06 17:11:29,None,0,2,RT @kdnuggets: Top 8 #Free Must-Read #Books on #DeepLearning #KDN https://t.co/1DtlN91Yjj
